In [3]:
import torch
import torch.nn as nn
from torchsummary import summary
from os.path import join
from tqdm import tqdm
import glob
import numpy as np
import matplotlib.patches as patches
import os

from modules.custom import Generator3, Generator4
from modules.pix2pix_networks import PixelDiscriminator
from modules.liteFlownet import lite_flownet as lite_flow
from config import Config,MyConfig
from init_utils import get_yolo
from ops import detect_nob, prep_frame
from matplotlib import pyplot as plt
import data_utils
import calc_scores
import crop_utils
import init_utils

In [2]:
def shape(tensor):
    return tensor.squeeze(0).shape

In [5]:
cfg = MyConfig()

for key in cfg.__dict__:
    print(key,':',cfg.__dict__[key])

gen_name = cfg.generator
fn_name = cfg.flownet
resume = cfg.resume
optimizer_name = cfg.optimizer
scheduler_name = cfg.scheduler
in_channels = 3
out_channels = 3

tr_name : shanghai_cropped_08
te_name : shanghaitech
device : cuda
cwd : /home/chojh21c/sunghyun/Anomaly_Detection/3D-Attention-based-Anomaly-Detection
ad_home : /home/chojh21c/sunghyun/Anomaly_Detection
data_home : /home/chojh21c/sunghyun/Anomaly_Detection/m8/datasets
save_name : checkpoints
nframe : 4
resize_h : 64
resize_w : 64
max_h : 480
max_w : 856
symmetric : False
factor_x : 1.5
factor_y : 1.2
confidence : 0.4
gamma : 0.3
generator : custom4
n_kernels : [32, 64, 64, 128]
bn_kernel : 128
flownet : lite
att_dim : 256
resume : None
batch_size : 32
epoch : 10
save_epoch : 5
verbose : 10
g_lr : 0.001
d_lr : 0.0001
l2 : 1e-05
init : kaiming
optimizer : adamw
scheduler : cosine
warm_cycle : 10000
lambdas : [1.0, 1.0, 1.0, 2.0]
show_curve : True
trained_model : None


생성기

In [6]:
generator = Generator4(in_channels, out_channels, cfg=cfg)
generator.to('cuda')

ckpt = torch.load('weight/Best/ped2/total_model_e14.pt')

generator.load_state_dict(ckpt['net_g'])

<All keys matched successfully>

In [91]:
input = torch.zeros(1,3,4,64,64).cuda() # B, C, D, H, W
output = generator(input)

print('input:', input.shape)
print('output:',output.shape)

print(summary(generator,shape(input)))
print(generator.parameters)

input: torch.Size([1, 3, 4, 64, 64])
output: torch.Size([1, 3, 64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 32, 4, 64, 64]           2,624
       BatchNorm3d-2        [-1, 32, 4, 64, 64]              64
              ReLU-3        [-1, 32, 4, 64, 64]               0
            Conv3d-4        [-1, 32, 4, 64, 64]          27,680
       BatchNorm3d-5        [-1, 32, 4, 64, 64]              64
              ReLU-6        [-1, 32, 4, 64, 64]               0
         MaxPool3d-7        [-1, 32, 4, 32, 32]               0
            Conv3d-8        [-1, 64, 4, 32, 32]          55,360
       BatchNorm3d-9        [-1, 64, 4, 32, 32]             128
             ReLU-10        [-1, 64, 4, 32, 32]               0
           Conv3d-11        [-1, 64, 4, 32, 32]         110,656
      BatchNorm3d-12        [-1, 64, 4, 32, 32]             128
             ReLU-13        [-1

플로우넷

In [48]:
flownet = lite_flow.Network()
flownet.to('cuda')

flownet.load_state_dict(
    torch.load('weight/network-default.pytorch')
)

<All keys matched successfully>

In [92]:
input1 = torch.zeros(1,3,64,64).cuda() # B, C, H, W
input2 = torch.zeros(1,3,64,64).cuda() # B, C, H, W
output = flownet(input1, input2)

print('input:', input1.shape, input2.shape)
print('output:',output.shape)

print(summary(flownet, [shape(input1), shape(input2)]))
print(flownet.parameters)

input: torch.Size([1, 3, 64, 64]) torch.Size([1, 3, 64, 64])
output: torch.Size([1, 2, 32, 32])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]           4,736
         LeakyReLU-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 32, 32, 32]           9,248
         LeakyReLU-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           9,248
         LeakyReLU-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 32, 32, 32]           9,248
         LeakyReLU-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 64, 16, 16]          18,496
        LeakyReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,928
        LeakyReLU-12           [-1, 64, 16, 16]               0
       

/home/chojh21c/anaconda3/envs/py36/lib/python3.9/site-packages/torch/nn/functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


판별기

In [86]:
discriminator = PixelDiscriminator(input_nc=out_channels)
discriminator.to('cuda')
discriminator.load_state_dict(ckpt['net_d'])

<All keys matched successfully>

In [93]:
input = torch.zeros(1,3,64,64).cuda() # B, C, H, W
output = discriminator(input)

print('input:', input.shape)
print('output:',output.shape)

print(summary(discriminator, shape(input)))
print(discriminator.parameters)

input: torch.Size([1, 3, 64, 64])
output: torch.Size([1, 1, 11, 11])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 33, 33]           6,272
         LeakyReLU-2          [-1, 128, 33, 33]               0
            Conv2d-3          [-1, 256, 17, 17]         524,544
         LeakyReLU-4          [-1, 256, 17, 17]               0
            Conv2d-5            [-1, 512, 9, 9]       2,097,664
         LeakyReLU-6            [-1, 512, 9, 9]               0
            Conv2d-7          [-1, 512, 10, 10]       4,194,816
         LeakyReLU-8          [-1, 512, 10, 10]               0
            Conv2d-9            [-1, 1, 11, 11]           8,193
Total params: 6,831,489
Trainable params: 6,831,489
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 4.67
Params size (MB): 26.06
E

객체 탐지

In [7]:
yolo_model = init_utils.get_yolo(cfg)
print(yolo_model)

Using cache found in /home/chojh21c/.cache/torch/hub/ultralytics_yolov5_master
requirements: tqdm>=4.41.0 not found and is required by YOLOv5, attempting auto-update...
requirements: 'pip install tqdm>=4.41.0' skipped (offline)
fatal: not a git repository (or any parent up to mount point /home)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
YOLOv5 🚀 2022-3-31 torch 1.11.0+cu113 CUDA:0 (Tesla V100-PCIE-32GB, 32511MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 64, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      